In [17]:
from __future__ import division
import gym
import torch
import random
import numpy as np
import torch
from PIL import Image
import torch
import torch.nn as nn
from collections import namedtuple
from collections import deque
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
from matplotlib.pyplot import imshow
from PIL import Image
from wrappers import make_atari, wrap_deepmind, wrap_pytorch
import queue
from torch import optim
import matplotlib.pyplot as plt

In [18]:
class SegmentSumTree:
    def __init__(self, capacity):
        self.size = capacity 
        #n leaves + n-1 internal = 2n-1, capacity of a tree
        self.tree = np.zeros((2*capacity-1), dtype = np.float32)
        #leaf nodes having actual values 
        self.transit_data_buffer =  np.array([None] * self.size) 
        self.is_full = False
        self.max_prior_val = 1
        self.num_entries = 0 
        self.idx = 0 
    
    
    def append(self, priority_val, transit_data):
        self.max_prior_val = max(priority_val, self.max_prior_val)
        self.transit_data_buffer[self.idx] = transit_data  
        self.update(self.idx + self.size - 1, priority_val) 
        self.idx = (self.idx + 1) % self.size 
        self.num_entries += 1
        self.is_full = self.is_full or self.idx == 0  
    
    
    def propagate(self, index, priority_val):
        parent_idx = self.getParentIdx(index)
        left_node_idx = 2 * parent_idx + 1
        right_node_idx = 2 * parent_idx + 2
        self.tree[parent_idx] = self.tree[left_node_idx] + self.tree[right_node_idx]
        if parent_idx != 0: 
            self.propagate(parent_idx, priority_val)
            
        
    def update(self, index, priority_val):
        self.tree[index] = priority_val
        self.max_prior_val = max(priority_val, self.max_prior_val)
        self.propagate(index, priority_val)
       
    
    def getParentIdx(self, index):
        return (index - 1)//2
        
            
        
    def search(self, value):
        idx = self.retrieve(0, value)  # Search for index of item from root
        data_index = idx - self.size + 1
        return (self.tree[idx], data_index, idx)
        
    
    def retrieve(self, idx, value):
        left_node_idx = 2 * idx + 1
        right_node_idx = 2 * idx + 2
        
        if left_node_idx >= len(self.tree):
            return idx
        
        elif value <= self.tree[left_node_idx]:
            return self.retrieve(left_node_idx, value)
            
        else:
            return self.retrieve(right_node_idx, value - self.tree[left_node_idx])
        
    
    def getNumEntries(self):
        return self.num_entries
     

    def getTotal(self):
        return self.tree[0]
        
    def getSize(self):
        return self.size
    
    
    def getMaxPriorValue(self):
        return self.max_prior_val
    
    
    def getPriorties(self):
        return self.priorities
    
    
    def getTree(self):
        return self.tree
    
    
    def getDataByIdx(self, idx):
        return self.transit_data_buffer[idx % self.size]
    

# Model

In [19]:
class QNet(torch.nn.Module):
    def __init__(self,obs_shape,act_shape):
        super(QNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(7*7*64,128)
        self.fc2 = nn.Linear(128,1)
        self.fc3 = nn.Linear(7*7*64,128)
        self.fc4 = nn.Linear(128,act_shape)
    def forward(self, x):
        #Conv
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0],-1)
        
        #Fc
        x1 = x
        x = self.fc1(x)
        x = self.relu(x)
        v = self.fc2(x)
        
        x1 = self.fc3(x1)
        x1 = self.relu(x1)
        adv = self.fc4(x1)
        
        q_s_a = v + adv - adv.mean()
        
        return q_s_a

In [20]:
def eps_greedy(epsilon,state,net):
    if(np.random.random()<epsilon):
        action = np.random.randint(ACT_SHAPE)
    else:
        qvalues = net(state)
        action = torch.argmax(qvalues).item()
    return action    

In [21]:
class ReplayBuffer(object):
    def __init__(self,maxsize):
        self.q = deque(maxlen = maxsize)
        self.maxsize = maxsize
    def add(self,x):
        self.q.append(x)
        if(len(self.q)==self.maxsize):
            self.q.popleft()
    def getSize(self):
        return len(self.q)
    def sample(self,size):
        batch = random.sample(list(self.q),size)
        state,action,reward,next_state,done = map(list, zip(*batch))
        return state,action,reward,next_state,done
           

In [124]:
Transition = namedtuple('Transition', ('timestep', 'state', 'action', 'reward', 'next_state','done'))
blank_trans = Transition(0, torch.zeros(84, 84, dtype=torch.uint8), None, 0, torch.zeros(84, 84, dtype=torch.uint8),False)

class PrioritizedReplayBuffer:
    def __init__(self,capacity = 1000000, batch_size = 32, gamma = 0.99, multi_step = 3, 
                beta = 0.4, alpha = 0.5):
        self.capacity = capacity
        self.discount = gamma
        self.n = multi_step
        self.beta = beta   
        self.alpha = alpha
        self.transit_buffer = SegmentSumTree(capacity)  
        self.prev_step_length = 4 #history_length
        self.timestep = 0  #t
        

    def add(self,state,action,reward,next_state,done):
#         state = state[-1].mul(255)  
#         next_state = next_state[-1].mul(255)
#         #print('coming here')
#         print('Before : ',self.transit_buffer.max_prior_val)
        self.transit_buffer.append(self.transit_buffer.max_prior_val, Transition(self.timestep, state, action, reward, next_state, done))  
#         print('After : ',self.transit_buffer.max_prior_val)
        
#         print('Tree : ',self.transit_buffer.tree)
#         print('Transitions : ',self.transit_buffer.transit_data_buffer)
#         print('-'*100)
        self.timestep = 0 if done else self.timestep + 1
        
        
    def sample(self,k): 
        batch, idxs, priorities = [], [], []
        root_total_priority = self.transit_buffer.getTotal()
        segment = root_total_priority / root_total_priority
        print('Root total priority',root_total_priority)
        
        priority_exponent_weight = (1 - self.beta) / (TMAX - TMIN)
        self.beta = np.min([1.0, self.beta + priority_exponent_weight])

        for i in range(k):
            a = segment * i
            b = segment * (i + 1)
            samp = random.uniform(a, b)
            priority, data_idx, idx = self.transit_buffer.search(samp)
            data = self.transit_buffer.getDataByIdx(data_idx)
            priorities.append(priority)
            batch.append(data)
            idxs.append(idx)
        sampling_probabilities = priorities / sum(priorities)
#         print("Probabilities : ",sampling_probabilities.sum())
        
        #compute importance sampling 
        weights = np.power(self.transit_buffer.getNumEntries() * sampling_probabilities, -self.beta)
        weights /= weights.max()
    
        timestep,states, actions, rewards, next_states, dones = zip(*batch)
        
#         print('-'*30)
#         print('length of states',len(states))
#         print('length of next states',len(next_states))
#         print('shape of states',states[0].shape)
#         print('shape of next states',next_states[0].shape)
        
        
        states, actions, rewards, next_states, dones = list(states), list(actions), list(rewards), \
                                                    list(next_states), list(dones) 
        states = torch.cat([x for x in states]).cuda()
        next_states = torch.cat([x for x in next_states]).cuda()
        actions = torch.Tensor(actions).long().cuda()
        dones = np.array(dones).astype(int)
        dones = torch.Tensor(dones).cuda()
        rewards = torch.Tensor(rewards).cuda()
        priorities = torch.Tensor(priorities).cuda()
        
#         print(states.shape)
#         print(next_states.shape)
        
        return priorities, idxs, states, actions, rewards, next_states, dones       
        
    def updatePriorities(self, indexes, priorities):
        priorities = np.power(priorities, self.alpha)
        for idx, priority_val in zip(indexes, priorities):
            self.transit_buffer.update(idx, priority_val)
        
        
    def getSize(self):
        return self.transit_buffer.getNumEntries()
        

# Loss function

In [125]:
def compute_loss(size):
    weights,idx,current_state,action,reward,next_state,done = buffer.sample(size)
    
#     print(current_state.shape)
#     print(action.shape)
#     print(reward.shape)
#     print(next_state.shape)
#     print(done.shape)
    
    qvalues = net(current_state)
    qvalues_next = net(next_state)
    target_net.eval()
    qvalues_target = target_net(next_state)
    
    #Q(s,a)
    q_a = qvalues.gather(1, action.unsqueeze(1)).squeeze(1)
    
    #Selecting action for target network
    selected = qvalues_next.max(1)[1]
    #Q'(s',argmax Q(s',a))
    q_a_target = qvalues_target.gather(1,selected.unsqueeze(1)).squeeze(1)
    
    #Computing target value
    target = reward + GAMMA * q_a_target * (1 - done)
    L = (target - q_a).pow(2)*weights
    
    print('priority : ',weights)
    new_priorities = L + edge_epsilon
    optimizer.zero_grad()
    
    L = L.mean()
    L.backward()
    optimizer.step()
    new_priorities = new_priorities.cpu().detach().numpy()
    buffer.updatePriorities(idx,new_priorities)
    print("Loss : ",L)
    
    
    return L

In [126]:
def update_target():
    target_net.load_state_dict(net.state_dict())

In [127]:
def epsilon_decay(ep):
    e = .01 + .99*np.exp(-ep/30000)
    return e

In [128]:
def addreward(id,item,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(item)+' '+'\n')
    f.close()

In [129]:
def addloss(id,loss,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(loss.item())+' '+'\n')
    f.close()

# Training

In [130]:
# env = gym.make('PongNoFrameskip-v4')
# env    = make_atari('Pong-v0')
# env    = make_atari('BankHeist-v0')
env    = make_atari('Pong-v0')
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

print(env.observation_space)
print(env.action_space)


Box(1, 84, 84)
Discrete(6)


In [131]:
net = QNet(env.observation_space.shape,env.action_space.n)
net = net.cuda()
target_net = QNet(env.observation_space.shape,env.action_space.n)
target_net = target_net.cuda()
update_target()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [132]:
ITERATIONS = 1000000
epsilon = .99
OBS_SHAPE = env.observation_space.shape
ACT_SHAPE = env.action_space.n
REPLAY_SAMPLE = 5000
BATCH_SIZE = 32
GAMMA = .99
T_upd = 1000
TMAX = 50e6
TMIN = 20e3
edge_epsilon = 1e-5
torch.manual_seed(1) 

In [133]:
# net.load_state_dict(torch.load('dqn-model.pth'))
# target_net.load_state_dict(torch.load('dqn-model-target.pth'))
# optimizer = optim.Adam(net.parameters(), lr=0.00001)
# update_target()

In [134]:
#buffer = ReplayBuffer(10000)
buffer = PrioritizedReplayBuffer(10000)
episode_reward = 0
state = env.reset()
state = torch.Tensor(state).cuda()
state = state.unsqueeze(0)
count = 0
# lossfile = './dueling-logs/losses.txt'
# rewardsfile = './dueling-logs/rewards.txt'
losses = []
rewards = [] 
for i in tqdm(range(ITERATIONS)):
    epsilon = epsilon_decay(i)
    action = eps_greedy(epsilon,state,net)
    next_state, reward, done, info = env.step(action)
    next_state = torch.Tensor(next_state).unsqueeze(0).cuda()
#     print('state input in buffer ',state.shape)
#     print('state input in buffer ',next_state.shape)
    
    buffer.add(state,action,reward,next_state,done)
    
    episode_reward+=reward
    state=next_state
    if(buffer.getSize()>REPLAY_SAMPLE):
        loss = compute_loss(BATCH_SIZE)
#         losses.append(loss)
#         if(i%500==0):
#             addloss(i,loss,lossfile)
    if done:
        count+=1
        rewards.append(episode_reward)
        print(episode_reward)
#         if(count%5==0):
#             addreward(count,episode_reward,rewardsfile)
#             print(episode_reward)
        episode_reward = 0 
        state = env.reset()
        state = torch.Tensor(state).cuda()
        state = state.unsqueeze(0)
    if(i%T_upd==0):
#         torch.save(net.state_dict(),'./dueling-logs/dqn-model.pth')
#         torch.save(target_net.state_dict(),'./dueling-logs/dqn-model-target.pth')
        update_target()


-20.0
-21.0
-20.0
Root total priority 5001.0
priority :  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')
Loss :  tensor(3.9482, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5024.4287
priority :  tensor([0.0544, 2.0863, 2.1219, 2.1219, 2.1219, 2.2442, 2.2442, 2.2442, 2.3028,
        2.3028, 2.9133, 2.9133, 2.9133, 2.3218, 2.3218, 2.3218, 2.1872, 2.1872,
        1.9394, 1.9394, 2.9365, 2.9365, 2.9365, 1.2022, 2.0056, 1.1915, 1.1915,
        3.0009, 3.0009, 3.0009, 1.0573, 2.2468], device='cuda:0')
Loss :  tensor(9.6797, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5037.4824
priority :  tensor([2.4832, 2.4832, 2.5581, 2.5581, 2.5581, 1.1266, 2.8761, 2.8761, 2.8761,
        3.0132, 3.0132, 3.0132, 4.2037, 4.2037, 4.2037, 4.2037, 4.2037, 3.0155,
        3.0155, 3.0155, 2.6825, 2.6825, 2.6825, 2.2152, 4.3260, 4.3260, 4.3260,
        4.3260

Loss :  tensor(12.2029, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5239.4395
priority :  tensor([3.4297, 3.4297, 3.4297, 3.4297, 1.1083, 1.1083, 2.1605, 2.1605, 1.2318,
        1.2318, 1.1454, 0.3004, 0.9869, 1.3241, 1.3241, 1.4692, 4.6423, 4.6423,
        4.6423, 4.6423, 4.6423, 4.6423, 4.6401, 4.6401, 4.6401, 4.6401, 5.6896,
        5.6896, 5.6896, 5.6896, 5.6896, 5.6896], device='cuda:0')
Loss :  tensor(16.8269, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5250.117
priority :  tensor([4.3851, 4.3851, 4.3851, 4.3851, 4.3851, 0.5870, 3.2342, 3.2342, 3.2342,
        3.2342, 0.4065, 2.3918, 2.3918, 0.4536, 0.2240, 0.7568, 0.8509, 4.8624,
        4.8624, 4.8624, 4.8624, 4.8624, 4.9181, 4.9181, 4.9181, 4.9181, 4.9181,
        5.6629, 5.6629, 5.6629, 5.6629, 5.6629], device='cuda:0')
Loss :  tensor(20.6028, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5261.468
priority :  tensor([0.1198, 5.1089, 5.1089, 5.1089, 5.1089, 5.1089, 4.0847, 4.

Loss :  tensor(1.3563, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5370.023
priority :  tensor([1.1147, 1.0288, 0.1256, 1.3642, 1.3642, 1.5144, 1.5144, 1.1617, 0.4121,
        0.9523, 1.4274, 1.4274, 1.1234, 0.6333, 1.2015, 1.2524, 1.2524, 0.1076,
        1.2699, 1.0930, 0.1960, 1.3176, 1.2673, 1.1538, 1.1538, 0.2810, 0.2237,
        1.4634, 1.5662, 0.2643, 0.7152, 0.3731], device='cuda:0')
Loss :  tensor(0.7742, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5374.2554
priority :  tensor([0.8614, 0.1791, 1.0289, 1.2013, 1.2013, 1.3443, 0.9059, 0.9059, 0.8343,
        1.2775, 1.2775, 0.4487, 1.0500, 1.0500, 0.9831, 0.0924, 0.8416, 0.8416,
        1.0435, 0.9945, 0.8568, 1.6123, 1.6123, 1.2129, 0.8255, 0.3274, 0.6864,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0')
Loss :  tensor(0.6235, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5378.615
priority :  tensor([0.7870, 0.7870, 1.0449, 1.0449, 1.1800, 0.4474, 0.6865, 0.713

priority :  tensor([0.5529, 0.2364, 0.0238, 0.1435, 0.4735, 0.0902, 0.0346, 0.4470, 1.0734,
        0.2982, 0.4354, 0.4281, 0.4281, 0.3720, 0.0731, 0.7612, 0.3512, 0.4361,
        0.5716, 0.2239, 0.3708, 0.2850, 0.1216, 0.4347, 0.5360, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0')
Loss :  tensor(0.1193, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5520.1484
priority :  tensor([0.3562, 0.2096, 0.0072, 0.3048, 0.0381, 0.0507, 0.2399, 1.0511, 0.0328,
        0.1912, 0.1921, 0.0248, 0.4014, 0.1008, 0.7142, 0.3457, 0.0940, 0.2867,
        0.1297, 1.0000, 0.4060, 0.2452, 0.4043, 0.5472, 0.0993, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0')
Loss :  tensor(0.1254, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5523.387
priority :  tensor([0.2851, 0.2991, 0.0522, 0.3422, 0.0343, 0.2253, 0.2469, 1.0310, 0.2462,
        0.2286, 0.0820, 0.2422, 0.4266, 0.4327, 0.3503, 0.0688, 0.1738, 0.0753,
   

Loss :  tensor(2.3577, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5756.5957
priority :  tensor([1.3570, 1.2549, 1.2549, 1.6137, 1.6278, 1.6278, 1.0933, 0.9840, 0.9840,
        1.0806, 1.5556, 0.3709, 1.1361, 0.1176, 1.6141, 1.6141, 1.8293, 1.8293,
        1.2509, 1.2509, 1.4749, 0.3766, 2.7753, 2.7753, 0.1997, 1.8533, 1.8533,
        0.3890, 2.0713, 2.0713, 2.0713, 1.3772], device='cuda:0')
Loss :  tensor(3.1162, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5772.2065
priority :  tensor([1.4976, 1.4976, 1.3672, 2.0264, 2.0264, 0.1890, 1.7329, 1.5185, 1.0682,
        1.0682, 1.7487, 1.7487, 1.7912, 1.7912, 0.8338, 0.2190, 1.2336, 1.7175,
        2.0263, 2.0263, 2.0263, 0.1965, 1.6282, 0.1707, 1.5901, 0.9525, 3.0956,
        3.0956, 3.0956, 0.0482, 2.1815, 2.1815], device='cuda:0')
Loss :  tensor(3.7406, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 5786.974
priority :  tensor([1.5829, 1.5829, 1.4353, 2.3280, 2.3280, 2.3280, 1.7975, 1.79

...

Loss :  tensor(0.0692, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22230.992
priority :  tensor([0.1135, 0.1063, 0.1308, 0.1609, 0.0740, 0.0841, 0.1356, 0.1194, 0.0091,
        0.1473, 0.9945, 0.0121, 0.1301, 0.1972, 0.5054, 0.0738, 0.1513, 0.4965,
        0.0857, 0.1614, 0.1332, 0.0644, 0.9223, 0.0610, 0.2355, 0.0033, 0.1177,
        0.0991, 0.0206, 0.1271, 0.1247, 0.0705], device='cuda:0')
Loss :  tensor(0.0858, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22245.412
priority :  tensor([0.1452, 0.1057, 0.0874, 0.1220, 0.4079, 0.1485, 0.1352, 0.1036, 0.0974,
        0.0628, 0.9877, 0.0403, 0.1453, 0.1088, 0.0219, 0.0461, 0.1348, 0.4981,
        0.1092, 0.0545, 0.0033, 0.1854, 0.9279, 0.0997, 0.1632, 0.4337, 0.1751,
        0.0437, 0.4471, 0.0074, 0.0886, 0.0751], device='cuda:0')
Loss :  tensor(0.0923, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22259.852
priority :  tensor([0.0870, 0.1068, 0.1669, 0.1809, 0.1088, 0.0543, 0.0941, 0.1

priority :  tensor([0.0813, 0.2255, 0.1312, 0.0907, 0.0411, 0.1540, 0.1314, 0.4156, 0.0939,
        0.1350, 0.1104, 0.9665, 0.1075, 0.1646, 0.1114, 0.0288, 0.0332, 0.1311,
        0.0838, 0.0597, 0.0982, 0.0960, 0.0751, 0.1245, 0.1425, 0.0809, 0.1132,
        0.1245, 0.1726, 0.0674, 0.4555, 0.0835], device='cuda:0')
Loss :  tensor(0.0534, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22577.145
priority :  tensor([0.0860, 0.1120, 0.0638, 0.1557, 0.0780, 0.1432, 0.0944, 0.1125, 0.0943,
        0.1324, 0.1363, 0.1353, 0.1550, 0.1221, 0.1013, 0.5153, 0.0234, 0.1224,
        0.1061, 0.1067, 0.0135, 0.1359, 0.0894, 0.1575, 0.9426, 0.1502, 0.2334,
        0.4512, 0.1236, 0.2200, 0.4566, 0.0953], device='cuda:0')
Loss :  tensor(0.0631, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22591.502
priority :  tensor([0.1266, 0.1135, 0.1308, 0.4113, 0.4113, 0.1460, 0.1065, 0.0436, 0.1086,
        0.0032, 0.1123, 0.1334, 0.0876, 0.0584, 0.1940, 0.1133, 0.1273, 0.0093,
  

priority :  tensor([0.1306, 0.0261, 0.2230, 0.1233, 0.0997, 0.0034, 0.1094, 0.1286, 0.0645,
        0.1370, 0.1370, 0.9904, 0.1061, 0.0246, 0.0746, 0.5169, 0.1073, 0.1246,
        0.1114, 0.4764, 0.0052, 0.1787, 0.3676, 0.0742, 0.9163, 0.1217, 0.1557,
        0.0551, 0.4578, 0.1687, 0.2193, 0.0600], device='cuda:0')
Loss :  tensor(0.0948, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22893.287
priority :  tensor([0.0804, 0.1110, 0.2187, 0.1477, 0.0032, 0.0319, 0.1426, 0.4028, 0.0152,
        0.0308, 0.1340, 0.9855, 0.0218, 0.1433, 0.1379, 0.0319, 0.5208, 0.1107,
        0.1347, 0.4778, 0.1619, 0.3713, 0.3713, 0.1319, 0.9192, 0.0455, 0.1508,
        0.2325, 0.4623, 0.1145, 0.1649, 0.1519], device='cuda:0')
Loss :  tensor(0.1046, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 22907.648
priority :  tensor([0.1328, 0.0989, 0.1049, 0.4158, 0.0623, 0.0563, 0.1118, 0.4017, 0.0900,
        0.0747, 0.1359, 0.9868, 0.1317, 0.0844, 0.1930, 0.0032, 0.1055, 0.1243,
  

Loss :  tensor(0.0459, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 23194.785
priority :  tensor([0.1477, 0.1231, 0.0048, 0.1023, 0.0579, 0.0703, 0.0966, 0.4016, 0.1077,
        0.0364, 0.1325, 0.1268, 0.1286, 0.0378, 0.1160, 0.0874, 0.1027, 0.1011,
        0.0810, 0.4741, 0.0470, 0.1525, 0.1562, 0.1320, 0.2012, 0.9212, 0.0908,
        0.0947, 0.2200, 0.4676, 0.1627, 0.0484], device='cuda:0')
Loss :  tensor(0.0574, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 23209.062
priority :  tensor([0.1258, 0.0978, 0.1238, 0.1121, 0.4255, 0.0554, 0.1072, 0.4046, 0.0876,
        0.1264, 0.0835, 0.9797, 0.0967, 0.1456, 0.1314, 0.0561, 0.5241, 0.0769,
        0.0854, 0.0627, 0.1014, 0.0773, 0.1189, 0.1986, 0.1175, 0.9258, 0.0949,
        0.0673, 0.2156, 0.4729, 0.0671, 0.2133], device='cuda:0')
Loss :  tensor(0.0944, device='cuda:0', grad_fn=<MeanBackward1>)
Root total priority 23223.373
priority :  tensor([0.0712, 0.0940, 0.1212, 0.4318, 0.0945, 0.0673, 0.1027, 0.0

KeyboardInterrupt: 